In [1]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
import pandas as pd
import numpy as np
from functions.application_runner import run_application
df = pd.read_csv("cleaned_zomato.csv")
df["cuisines"] = df["cuisines"].apply(eval)


In [13]:
import pandas as pd
import numpy as np
import random

# Type of cuisines:
def cuisines_list(df:pd.DataFrame):
       
    global dict_of_cuisines
    dict_of_cuisines = {}
    for x in df.index:
        for y in df.loc[x,"cuisines"]:
            if y not in dict_of_cuisines.keys():
                dict_of_cuisines[y] = 1
            else:
                dict_of_cuisines[y] += 1
    dict_of_cuisines = dict(sorted(dict_of_cuisines.items(), key=lambda item: item[1], reverse=True))
    dict_of_cuisines["All"] = 0 
    return dict_of_cuisines


In [2]:

def display_widgets(df:pd.DataFrame):

    cuisine_selection = widgets.SelectMultiple(description='Cuisine type: ', 
                                                options=cuisines_list(df).keys(),
                                                value=('All',),
                                                disabled=False)
    price_selection = widgets.SelectionRangeSlider(options=['$', '$$', '$$$','$$$$'],
                                                    index=(0, 3), 
                                                    description='Price range: ',
                                                    disabled=False)
    rate_selection = widgets.FloatRangeSlider(description='Rating: ', min=1.8, 
                                                max= 5.0, step=0.1, disabled=False)
    new_restaurants = widgets.Checkbox(value=False, description='Include new restaurants: ')
    
    book_table = widgets.Checkbox(value=False, description='Include new restaurants: ')
    online_order = widgets.Checkbox(value=False, description='Include new restaurants: ')

    out = widgets.Output()
    button = widgets.Button(description="OK")
    
    global preferences
    preferences={}


    def collect_preferences():

        preferences["cuisine"] = cuisine_selection.value
        preferences["rate"] = rate_selection.value
        preferences["price"] = price_selection.value
        preferences["new_restaurants"] = new_restaurants.value
        return preferences

    def display_preferences(preferences):
        print(f"""Your preferences:

        -- selected cuisine types: {cuisine_selection.value} 
        -- selected price range: {price_selection.value[0]} - {price_selection.value[0]}
        -- selected rate range: {rate_selection.value[0]} - {rate_selection.value[1]}
        -- include new restaurants: {preferences["new_restaurants"]}""")

    def on_button_clicked(b):
        
        collect_preferences()
        cuisine_selection.close()
        price_selection.close()
        rate_selection.close()
        new_restaurants.close()
        out.close()
        button.close()
        display_preferences(preferences)
        run_application(preferences,df)
        
    

    @out.capture()
    def changed(b):
        out.clear_output()
        display_preferences(preferences)
        collect_preferences()
        # run_application(preferences,df)
        
    cuisine_selection.observe(changed)
    price_selection.observe(changed)
    rate_selection.observe(changed)
    new_restaurants.observe(changed)

    display(cuisine_selection,price_selection,rate_selection, new_restaurants,button)
    button.on_click(on_button_clicked)
    

In [3]:
display_widgets(df)

NameError: name 'cuisines_list' is not defined

In [ ]:
# Live data filter

In [4]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
import pandas as pd
import numpy as np
df = pd.read_csv("cleaned_zomato.csv")
df["cuisines"] = df["cuisines"].apply(eval)

In [5]:
# Type of cuisines:
def cuisines_list(df:pd.DataFrame):
       
    global dict_of_cuisines
    dict_of_cuisines = {}
    for x in df.index:
        for y in df.loc[x,"cuisines"]:
            if y not in dict_of_cuisines.keys():
                dict_of_cuisines[y] = 1
            else:
                dict_of_cuisines[y] += 1
    dict_of_cuisines = dict(sorted(dict_of_cuisines.items(), key=lambda item: item[1], reverse=True))
    dict_of_cuisines["All"] = 0 
    return dict_of_cuisines

In [11]:
# Price getter:

def get_price(price_choice:tuple):

    p=price_choice
    q25 = df["cost"].quantile(0.25)
    q50 = df["cost"].quantile(0.5)
    q75 = df["cost"].quantile(0.75)
    max = df["cost"].max()
    price_ranges={'$':(0,q25), '$$': (q25,q50), '$$$':(q50,q75), '$$$$':(q75,max+1)}
    price_min = float(price_ranges[p[0]][0])
    price_max = float(price_ranges[p[1]][1])

    return price_min,price_max
    

In [171]:
cuisine_selection = widgets.SelectMultiple(description='Cuisine type: ', 
                                            options=cuisines_list(df).keys(),
                                            value=('All',),
                                            disabled=False)
output_cuisine = widgets.Output()

def cuisine_handler(change):
    output_cuisine.clear_output()
    with output_cuisine:
        if (change.new == ('All',)):
            display(df)
        else:
            c = change.new
            condition = df["cuisines"].apply(lambda x: x if any(z in c for z in x) else None).notnull()
            display(df[condition])

cuisine_selection.observe(cuisine_handler, names='value')
display(cuisine_selection)

SelectMultiple(description='Cuisine type: ', index=(107,), options=('North Indian', 'Chinese', 'South Indian',…

In [173]:
display(output_cuisine)

Output()

In [8]:
from functions.application_runner import run_application
df = pd.read_csv("cleaned_zomato.csv")
df["cuisines"] = df["cuisines"].apply(eval)
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
def display_interactive_widgets(df):
    
    cuisine_selection = widgets.SelectMultiple(description='Cuisine type: ', 
                                            options=cuisines_list(df).keys(),
                                            value=('All',),
                                            disabled=False)

    price_selection = widgets.SelectionRangeSlider(options=['$', '$$', '$$$','$$$$'],
                                                index=(0, 3), 
                                                description='Price range: ',
                                                disabled=False)

    rate_selection = widgets.FloatRangeSlider(description='Rating: ', min=1.8, 
                                            max= 5.0, step=0.1, disabled=False)

    new_rest = widgets.Checkbox(value=False, description='Include new restaurants')
    
    button = widgets.Button(description="OK")
    box_layout = widgets.Layout(
                align_items= 'center')

    outputf = widgets.Output()
    plot_output = widgets.Output()
    stat_output = widgets.Output()
    map_output = widgets.Output()

    global preferences
    preferences = {}
    def collect_preferences(c,r,p,n):

        preferences["cuisine"] = c
        preferences["rate"] = r
        preferences["price"] = p
        preferences["new_restaurants"] = n
        return preferences

    def display_preferences(preferences):
        print(f"""Your preferences:

        -- selected cuisine types: {cuisine_selection.value} 
        -- selected price range: {price_selection.value[0]} - {price_selection.value[1]}
        -- selected rate range: {rate_selection.value[0]} - {rate_selection.value[1]}
        -- include new restaurants: {preferences["new_restaurants"]}""")

    def on_button_clicked(b):

        with map_output:
            map_output.clear_output()
            display_preferences(preferences)
            df_to_process = df.copy()
            run_application(preferences,df_to_process)


    def common_filtering(cuisine, price, rate, new_restaurants):

        outputf.clear_output()
        plot_output.clear_output()
        stat_output.clear_output()

        c = cuisine_selection.value
        r = rate_selection.value
        p = price_selection.value
        n = new_rest.value

        collect_preferences(c,r,p,n)
        

        if c != ("All",):
            condition = df["cuisines"].apply(lambda x: x if any(z in c for z in x) else None).notnull()
        else:
            condition = df["cuisines"].notnull()

        price_min, price_max = get_price(p)

        if n == True:
            common_filter = df[condition].query("@r[0] < rate <= @r[1] or rate.isnull() and @price_min <= cost < @price_max or cost.isnull()")
        else:
            common_filter = df[condition].query("@r[0] < rate <= @r[1] and @price_min <= cost < @price_max")


        def calculate_statistics(df:pd.DataFrame):

            print(f"""Your preferences:
            -- mean cost of restaurants: {df['rate'].mean()} 
            -- number of restaurants: {df['rate'].count()}
            -- correlation cost x votes: {df['cost'].mean()}
            -- correlation cost x rate: {df['cost'].mean()} """)

        
        with outputf:
            display_preferences(preferences)
            display(common_filter)
            
        with plot_output:
            display_preferences(preferences)
            # sns.pairplot(common_filter['cost','rate', 'votes'], shade=True)
            # sns.pairplot(common_filter,x_vars=['cost','rate','votes'],
            #             y_vars=['cost','rate','votes'],)
            lab = ['cost', 'votes', 'rate']
            plt.boxplot([common_filter['cost'], common_filter['votes'], common_filter['rate']],labels=lab, showmeans=True)
            # plt.hist(common_filter['cost'])
            plt.show()
        with stat_output:
            calculate_statistics(common_filter)

    def cuisine_eventhandler(change):
        common_filtering(change.new, price_selection.value, rate_selection.value, new_rest.value)

    def price_eventhandler(change):
        common_filtering(cuisine_selection.value, change.new, rate_selection.value, new_rest.value)

    def rate_eventhandler(change):
        common_filtering(cuisine_selection.value, price_selection.value, change.new, new_rest.value)

    def new_restaurants_eventhandler(change):
        common_filtering(cuisine_selection.value, price_selection.value, rate_selection.value, change.new)

    cuisine_selection.observe(cuisine_eventhandler, names='value')
    price_selection.observe(price_eventhandler, names='value')
    rate_selection.observe(rate_eventhandler, names='value')
    new_rest.observe(new_restaurants_eventhandler)

    item_layout = widgets.Layout(margin='0 0 50px 0')
    input_widgets = widgets.HBox([cuisine_selection,price_selection,rate_selection,new_rest])
    button_layout = widgets.VBox([button],layout=box_layout)
    tab = widgets.Tab([outputf, plot_output, stat_output,map_output],layout=item_layout)
    tab.set_title(0, 'Dataset')
    tab.set_title(1, 'Plot')
    tab.set_title(2, 'Statistics')
    tab.set_title(3, 'Application output')
    dashboard = widgets.VBox([input_widgets,button_layout,tab],layout=item_layout)
    display(dashboard)
    button.on_click(on_button_clicked)

In [13]:
display_interactive_widgets(df)